### Job Status and Control API

This notebook demonstrates how to use the `FluxExecutor` class to submit job bundles, get event updates, and wait until all the submitted jobs complete.

First, import the necessary modules and determine the number of nodes and cores available to your allocation.

In [ ]:
import os
from flux.job import JobspecV1, FluxExecutor

ncores = !flux resource list -no {ncores} --state=up
nc = int(ncores[0])

nnodes = !flux resource list -no {nnodes} --state=up
nn = int(nnodes[0])

Create an _event callback_ you will attach to each submitted job.

In [ ]:
def event_callback(future, event):
    print(f"job {future.jobid()} triggered event {event.name!r}")

Create a pair of jobs, each with different resoruces requirements.

In [ ]:
compute_jobreq = JobspecV1.from_command(
        command=["./flux-workflow-examples/job-status-control/compute.py", "10"], num_tasks=nn*2, num_nodes=nn, cores_per_task=2
    )
compute_jobreq.cwd = os.getcwd()
compute_jobreq.environment = dict(os.environ)

io_jobreq = JobspecV1.from_command(
    command=["./flux-workflow-examples/job-status-control/io-forwarding.py", "10"], num_tasks=nn, num_nodes=nn, cores_per_task=1
)
io_jobreq.cwd = os.getcwd()
io_jobreq.environment = dict(os.environ)

Finally, submit the jobs and observe the event lifecycle as _flux_ schedules and executes each submitted job.

In [ ]:
njobs = 12

with FluxExecutor() as executor:
    futures = [executor.submit(compute_jobreq) for _ in range(njobs // 2)]
    futures.extend(
        executor.submit(io_jobreq) for _ in range(njobs // 2, njobs)
    )
    print("bookkeeper: all jobs submitted")
    for fut in futures:
        # each event can have a different callback
        for event in executor.EVENTS:
            fut.add_event_callback(event, event_callback)
            
    print("bookkeeper: waiting until all jobs complete")
    
# exiting the context manager waits for the executor to complete all futures
print("bookkeeper: all jobs completed")